In [ ]:
#############################################################################
# zlib License
#
# (C) 2023 Murtaza Safdari <musafdar@cern.ch>, Jongho Lee <jongho.lee@cern.ch>
#
# This software is provided 'as-is', without any express or implied
# warranty.  In no event will the authors be held liable for any damages
# arising from the use of this software.
#
# Permission is granted to anyone to use this software for any purpose,
# including commercial applications, and to alter it and redistribute it
# freely, subject to the following restrictions:
#
# 1. The origin of this software must not be misrepresented; you must not
#    claim that you wrote the original software. If you use this software
#    in a product, an acknowledgment in the product documentation would be
#    appreciated but is not required.
# 2. Altered source versions must be plainly marked as such, and must not be
#    misrepresented as being the original software.
# 3. This notice may not be removed or altered from any source distribution.
#############################################################################

In [ ]:
from beamtest_analysis_helper import etroc2_analysis_helper
import datetime
from pathlib import Path
import pandas as pd
import numpy as np
from glob import glob
from natsort import natsorted
import hist
import mplhep as hep
hep.style.use('CMS')

In [ ]:
# !!!!!!!!!!!!
# It is very important to correctly set the chip name, this value is stored with the data

chip_names = ["ET2_W36_IP7_13_HV210V_offset24","ET2_EPIR_1_1_HV210V_offset24", "ET2_CNM_1_3_HV210V_offset6"]
chip_fignames = chip_names
chip_figtitles = ["ETROC2 WB W36 IP7-13 HV210V OS:24","(Trigger) ETROC2 BB EPIR 1-1 HV210V OS:24", "ETROC2 BB CNM 1-3 HV210V OS:6"]

chip_labels= ["1","0","3"]

today = datetime.date.today().isoformat()
fig_outdir = Path('../../ETROC-figures')
fig_outdir = fig_outdir / (today + '_Array_Test_Results')
fig_outdir.mkdir(exist_ok=True)
fig_path = str(fig_outdir)

# path_pattern = f"*2023-09-21_Array_Test_Results/SelfTrigger_bottom_Readout_topbottom_1"
# path_pattern = f"./testbeam_sep24/SelfTrigger_ET2_CNM_BATCH_1_3_Readout_ET2_EPIR_BATCH1_1_ET2_W36_IP7_13_ET2_CNM_BATCH1_3_loop_*.pqt"
path_pattern = "./highpower_offset6/SelfTrigger_ET2_EPIR_BATCH1_1_Readout_ET2_W36_IP7_13_ET2_CNM_BATCH1_3_offset6_highpower_FINALRUN_loop_*.pqt"

helper = etroc2_analysis_helper()

In [ ]:
#%%
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.patches as mpatch

fig = plt.figure(dpi=50, figsize=(5,5))
gs = fig.add_gridspec(1,1)

ax0 = fig.add_subplot(gs[0,0])
ax0.plot([1, 0], [1, 0])
plt.show()

## Event selections

In [ ]:
files = glob(path_pattern)
files = natsorted(files)

dataframes = []

for ifile in files:
    tmp_df = pd.read_parquet(ifile)
    if tmp_df.empty:
        continue

    # Group the DataFrame by 'evt' and count unique 'board' values in each group
    unique_board_counts = tmp_df.groupby('evt')['board'].nunique()

    ## event has three unique board ID
    event_numbers_with_three_unique_boards = unique_board_counts[unique_board_counts == 3].index
    subset_df = tmp_df[tmp_df['evt'].isin(event_numbers_with_three_unique_boards)]
    subset_df.reset_index(inplace=True, drop=True)

    del tmp_df
    if subset_df.empty: 
        continue
    
    dataframes.append(subset_df)

df = pd.concat(dataframes)
df.reset_index(inplace=True, drop=True)
del dataframes

In [ ]:
df.info()

In [ ]:
h_inclusive = helper.return_hist(df, chip_names, chip_labels)

In [ ]:
helper.make_pix_inclusive_plots(h_inclusive, chip_names[0], chip_fignames[0], chip_figtitles[0], fig_path, save=False, show=True, tag="inclusive", title_tag=", inclusive")

In [ ]:
helper.make_pix_inclusive_plots(h_inclusive, chip_names[1], chip_fignames[1], chip_figtitles[1], fig_path, save=False, show=True, tag="inclusive", title_tag=", inclusive")

In [ ]:
helper.make_pix_inclusive_plots(h_inclusive, chip_names[2], chip_fignames[2], chip_figtitles[2], fig_path, save=False, show=True, tag="inclusive", title_tag=", inclusive")

In [ ]:
del h_inclusive

In [ ]:
helper.making_heatmap_byPandas(df, chip_labels, chip_figtitles, "inclusive")

In [ ]:
helper.making_3d_heatmap_byPandas(df, chip_labels, chip_figtitles, "inclusive")

### Event counter and 2D heatmap based on WB pixel selection

In [ ]:
pix_info = [1, 15, 6]
simple_filtered_df = helper.find_maximum_event_combination(df, pix_info)

In [ ]:
helper.making_heatmap_byPandas(simple_filtered_df, chip_labels, chip_figtitles, figtitle_tag="WB pixel (15, 6)")

In [ ]:
del simple_filtered_df

### Pixel ID selection based on the event counter

In [ ]:
pix_dict = {
    # board ID: [row, col]
    0: [ 2, 6],
    1: [15, 6],
    2: [ 0, 0],
    3: [ 3, 5],
}

filtered_group = helper.pixel_filter(df, pix_dict)
filtered_group = helper.singlehit_event_clear_func(filtered_group)
filtered_group.info()

In [ ]:
del df

In [ ]:
h_pix_selected = helper.return_hist(filtered_group, chip_names, chip_labels)

In [ ]:
helper.make_pix_inclusive_plots(h_pix_selected, chip_names[0], chip_fignames[0], chip_figtitles[0], fig_path, save=False, show=True, tag="inclusive", title_tag=", Pixel (15, 6)")

In [ ]:
helper.make_pix_inclusive_plots(h_pix_selected, chip_names[1], chip_fignames[1], chip_figtitles[1], fig_path, save=False, show=True, tag="inclusive", title_tag=", Pixel (2, 6)")

In [ ]:
helper.make_pix_inclusive_plots(h_pix_selected, chip_names[2], chip_fignames[2], chip_figtitles[2], fig_path, save=False, show=True, tag="inclusive", title_tag=", Pixel (3, 5)")

In [ ]:
del h_pix_selected

### TDC cut

In [ ]:
# Define custom filtering criteria for each board
tdc_cuts = {    
    # board ID: [CAL LB, CAL UB, TOA LB, TOA UB, TOT LB, TOT UB]
    0: [199, 205,   0, 1100,   0, 600],
    1: [200, 210, 275,  350,   0, 600],
    2: [160, 220,   0, 1100,   0, 600],
    3: [189, 195,   0, 1000,   0, 600],
}

tdc_filtered_df = helper.tdc_event_selection(filtered_group, tdc_cuts)
tdc_filtered_df = helper.singlehit_event_clear_func(tdc_filtered_df)
tdc_filtered_df.info()

In [ ]:
del filtered_group

In [ ]:
def sort_filter(group):
    return group.sort_values(by=['board'], ascending=True)

def distance_filter(group, distance):
    board0_row = group[(group["board"] == 0)]
    board3_row = group[(group["board"] == 3)]
    board0_col = group[(group["board"] == 0)]
    board3_col = group[(group["board"] == 3)]

    if not board0_row.empty and not board3_row.empty and not board0_col.empty and not board3_col.empty:
        row_index_diff = abs(board0_row["row"].values[0] - board3_row["row"].values[0])
        col_index_diff = abs(board0_col["col"].values[0] - board3_col["col"].values[0])
        return row_index_diff < distance and col_index_diff < distance
    else:
        return False
    
# tmp_group = selected_subset_df.groupby('evt')
# filtered_simple_group = tmp_group.filter(simple_filter, board=1, row=15, col=6)
# filtered_simple_group.reset_index(inplace=True, drop=True)
# del tmp_group

# grouped = filtered_simple_group.groupby('evt')
# sorted_filtered_simple_group = grouped.apply(sort_filter)
# sorted_filtered_simple_group.reset_index(inplace=True, drop=True)
# sorted_filtered_simple_group
# del grouped

# grouped = sorted_filtered_simple_group.groupby('evt')
# dis_simple_group = grouped.filter(distance_filter, distance=2)
# dis_simple_group

# test_group = dis_simple_group.groupby(['board', 'row', 'col'])
# test = test_group.size().reset_index(name='count')
# test.to_csv('test.csv', index=False)

# del filtered_simple_group,sorted_filtered_simple_group,grouped,dis_simple_group, test_group

In [ ]:
h_tdc_selection = helper.return_hist(tdc_filtered_df, chip_names, chip_labels)

In [ ]:
helper.make_pix_inclusive_plots(h_tdc_selection, chip_names[0], chip_fignames[0], chip_figtitles[0], fig_path, save=False, show=True, tag="after_tdc_cut", title_tag=", Pixel (15, 6) after TDC cut")

In [ ]:
helper.make_pix_inclusive_plots(h_tdc_selection, chip_names[1], chip_fignames[1], chip_figtitles[1], fig_path, save=False, show=True, tag="", title_tag=", Pixel (2, 6) after TDC cut")

In [ ]:
helper.make_pix_inclusive_plots(h_tdc_selection, chip_names[2], chip_fignames[2], chip_figtitles[2], fig_path, save=False, show=True, tag="", title_tag=", Pixel (2, 5) after TDC cut")

In [ ]:
del h_tdc_selection

## Convert TDC code to TDC time in [ns]

In [ ]:
selected_df = tdc_filtered_df

pix_rows = []
pix_cols = []
fit_params = []
cal_means = {boardID:{} for boardID in chip_labels}

for boardID in chip_labels:
    groups = selected_df[selected_df['board'] == int(boardID)].groupby(['row', 'col'])
    for (row, col), group in groups:
        
        cal_mean = group['cal'].mean()
        cal_means[boardID][(row, col)] = cal_mean

cal_means

In [ ]:
bin0 = (3.125/cal_means["0"][(2, 6)])
bin1 = (3.125/cal_means["1"][(15, 6)])
bin3 = (3.125/cal_means["3"][(3, 5)])

toa_in_time_b0 = 12.5 - selected_df[selected_df['board'] == 0]['toa'] * bin0
toa_in_time_b1 = 12.5 - selected_df[selected_df['board'] == 1]['toa'] * bin1
toa_in_time_b3 = 12.5 - selected_df[selected_df['board'] == 3]['toa'] * bin3

tot_in_time_b0 = (2*selected_df[selected_df['board'] == 0]['tot'] - np.floor(selected_df[selected_df['board'] == 0]['tot']/32)) * bin0
tot_in_time_b1 = (2*selected_df[selected_df['board'] == 1]['tot'] - np.floor(selected_df[selected_df['board'] == 1]['tot']/32)) * bin1
tot_in_time_b3 = (2*selected_df[selected_df['board'] == 3]['tot'] - np.floor(selected_df[selected_df['board'] == 3]['tot']/32)) * bin3

d = {
    'evt': selected_df['evt'].unique(),
    'toa_b0': toa_in_time_b0.to_numpy(),
    'tot_b0': tot_in_time_b0.to_numpy(),
    'toa_b1': toa_in_time_b1.to_numpy(),
    'tot_b1': tot_in_time_b1.to_numpy(),
    'toa_b3': toa_in_time_b3.to_numpy(),
    'tot_b3': tot_in_time_b3.to_numpy(),
}

df_in_time = pd.DataFrame(data=d)
del d, selected_df, tdc_filtered_df

df_in_time.info()

## Traditional Time Walk Correction

In [ ]:
del_toa_b0 = (0.5*(df_in_time['toa_b1'] + df_in_time['toa_b3']) - df_in_time['toa_b0']).values
del_toa_b1 = (0.5*(df_in_time['toa_b0'] + df_in_time['toa_b3']) - df_in_time['toa_b1']).values
del_toa_b3 = (0.5*(df_in_time['toa_b0'] + df_in_time['toa_b1']) - df_in_time['toa_b3']).values

In [ ]:
coeff_b0 = np.polyfit(df_in_time['tot_b0'].values, del_toa_b0, 3)
poly_func_b0 = np.poly1d(coeff_b0)

coeff_b1 = np.polyfit(df_in_time['tot_b1'].values, del_toa_b1, 3)
poly_func_b1 = np.poly1d(coeff_b1)

coeff_b3 = np.polyfit(df_in_time['tot_b3'].values, del_toa_b3, 3)
poly_func_b3 = np.poly1d(coeff_b3)

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(30, 10))
hep.cms.text(loc=0, ax=axes[0], text="Preliminary", fontsize=25)
axes[0].set_title(f'Board 0 Time Walk Correction', loc="right", size=25)
axes[0].scatter(df_in_time['tot_b0'].values,  del_toa_b0, label='data')
axes[0].plot(df_in_time['tot_b0'].values, poly_func_b0(df_in_time['tot_b0'].values), 'r.', label='fit')
axes[0].set_xlabel('TOT time [ns]')
axes[0].set_ylabel(r'$(TOA_{i} + TOA_{j})/2 - TOA$ [ns]' )
axes[0].legend()

hep.cms.text(loc=0, ax=axes[1], text="Preliminary", fontsize=25)    
axes[1].set_title(f'Board 1 Time Walk Correction', loc="right", size=25)
axes[1].scatter(df_in_time['tot_b1'].values,  del_toa_b1, label='data')
axes[1].plot(df_in_time['tot_b1'].values, poly_func_b1(df_in_time['tot_b1'].values), 'r.', label='fit')
axes[1].set_xlabel('TOT time [ns]')
axes[1].set_ylabel(r'$(TOA_{i} + TOA_{j})/2 - TOA$ [ns]')
axes[1].legend()

hep.cms.text(loc=0, ax=axes[2], text="Preliminary", fontsize=25)
axes[2].set_title(f'Board 3 Time Walk Correction', loc="right", size=25)
axes[2].scatter(df_in_time['tot_b3'].values,  del_toa_b3, label='data')
axes[2].plot(df_in_time['tot_b3'].values, poly_func_b3(df_in_time['tot_b3'].values), 'r.', label='fit')
axes[2].set_xlabel('TOT time [ns]')
axes[2].set_ylabel(r'$(TOA_{i} + TOA_{j})/2 - TOA$ [ns]')
axes[2].legend()

plt.tight_layout()

In [ ]:
def poly_3d_fit(x, y, a, b, c, d, e, f, g, h, i, j):
    return a*x**3 + b*y**3 + c*x**2*y + d*x*y**2 + e*x**2 + f*y**2 + g*x*y + h*x + i*y + j

def _poly_3d_fit(M, *args):
    x, y = M
    arr = np.zeros(x.shape)
    for i in range(len(args)//10):
       arr += poly_3d_fit(x, y, *args[i*10:i*10+10])
    return arr

def get_basis(x, y, max_order=3):
    """Return the fit basis polynomials: 1, x, x^2, ..., xy, x^2y, ... etc."""
    basis = []
    for i in range(max_order+1):
        for j in range(max_order - i +1):
            basis.append(x**j * y**i)
    return basis

X = df_in_time['tot_b0'].ravel()
Y = df_in_time['tot_b1'].ravel()
Z = (df_in_time['toa_b0'] - df_in_time['toa_b1']).ravel()

xmin = X.min()
xmax = X.max()
ymin = Y.min()
ymax = Y.max()
nx = 100
ny = 100

x, y = np.linspace(xmin, xmax, nx), np.linspace(ymin, ymax, ny)

max_order = 3   

In [ ]:
basis = get_basis(X, Y, max_order)
A = np.vstack(basis).T
b = Z.ravel()
c, r, rank, s = np.linalg.lstsq(A, b, rcond=None)

In [ ]:
print('Fitted parameters:')
print(c)

In [ ]:
# Calculate the fitted surface from the coefficients, c.
fit = np.sum(c[:, None, None] * np.array(basis)
                .reshape(len(basis), *X.shape), axis=0)


In [ ]:
fit
c[:, None, None]
c[:, None, None]* np.array(basis)
basis
np.array(basis)
c[:, None, None]* np.array(basis)
(c[:, None, None]* np.array(basis)).shape
c[:, None, None]* np.array(basis)
fit = np.sum(c[:, None, None] * np.array(basis)
                .reshape(len(basis), *X.shape), axis=0)
fit
np.array(basis)
c[:, None, None]

In [ ]:
c[:, None, None]* np.array(basis)

In [ ]:
(c[:, None, None]* np.array(basis)
  .reshape(len(basis), *X.shape))

In [ ]:
c.sum()

In [ ]:
fit[1]

In [ ]:
# Plot the 3D figure of the fitted function and the residuals.
fig = plt.figure()
ax = fig.add_subplot(projection='3d')
meshX, meshY = np.meshgrid(X, Y)
ax.plot_surface(meshX, meshY, fit, cmap='viridis')
#cset = ax.contourf(X, Y, Z-fit, zdir='z', offset=-4, cmap='viridis')
ax.set_zlim(-4,np.max(fit))
plt.show()

# Plot the test data as a 2D image and the fit as overlaid contours.
# fig = plt.figure()
# ax = fig.add_subplot(111)
# ax.imshow(Z, origin='lower', cmap='viridis',
#           extent=(x.min(), x.max(), y.min(), y.max()))
# ax.contour(X, Y, fit, colors='w')
# plt.show()

In [ ]:
corr_toas = helper.iterative_timewalk_correction(df_in_time, 5, 3)

In [ ]:
nth_del_toa_b0 = (0.5*(corr_toas[1] + corr_toas[2]) - corr_toas[0])
nth_del_toa_b1 = (0.5*(corr_toas[0] + corr_toas[2]) - corr_toas[1])
nth_del_toa_b3 = (0.5*(corr_toas[0] + corr_toas[1]) - corr_toas[2])

In [ ]:
coeff_b0 = np.polyfit(df_in_time['tot_b0'].values, nth_del_toa_b0, 3)
poly_func_b0 = np.poly1d(coeff_b0)

coeff_b1 = np.polyfit(df_in_time['tot_b1'].values, nth_del_toa_b1, 3)
poly_func_b1 = np.poly1d(coeff_b1)

coeff_b3 = np.polyfit(df_in_time['tot_b3'].values, nth_del_toa_b3, 3)
poly_func_b3 = np.poly1d(coeff_b3)

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(30, 10))
hep.cms.text(loc=0, ax=axes[0], text="Preliminary", fontsize=25)
axes[0].set_title(f'Board 0 Time Walk Correction', loc="right", size=25)
axes[0].scatter(df_in_time['tot_b0'].values,  nth_del_toa_b0, label='data')
axes[0].plot(df_in_time['tot_b0'].values, poly_func_b0(df_in_time['tot_b0'].values), 'r.', label='fit')
axes[0].set_xlabel('TOT time [ns]')
axes[0].set_ylabel(r'$(TOA_{i} + TOA_{j})/2 - TOA$ [ns]' )
axes[0].legend() 

hep.cms.text(loc=0, ax=axes[1], text="Preliminary", fontsize=25)    
axes[1].set_title(f'Board 1 Time Walk Correction', loc="right", size=25)
axes[1].scatter(df_in_time['tot_b1'].values,  nth_del_toa_b1, label='data')
axes[1].plot(df_in_time['tot_b1'].values, poly_func_b1(df_in_time['tot_b1'].values), 'r.', label='fit')
axes[1].set_xlabel('TOT time [ns]')
axes[1].set_ylabel(r'$(TOA_{i} + TOA_{j})/2 - TOA$ [ns]')
axes[1].legend()

hep.cms.text(loc=0, ax=axes[2], text="Preliminary", fontsize=25)
axes[2].set_title(f'Board 3 Time Walk Correction', loc="right", size=25)
axes[2].scatter(df_in_time['tot_b3'].values,  nth_del_toa_b3, label='data')
axes[2].plot(df_in_time['tot_b3'].values, poly_func_b3(df_in_time['tot_b3'].values), 'r.', label='fit')
axes[2].set_xlabel('TOT time [ns]')
axes[2].set_ylabel(r'$(TOA_{i} + TOA_{j})/2 - TOA$ [ns]')
axes[2].legend()

plt.tight_layout()

In [ ]:
corrTOA_b0 = hist.Hist(hist.axis.Regular(60, 0, 12, name="TWC_TOA", label=r'Time Walk Corrected $\Delta$TOA [ns]'))
corrTOA_b1 = hist.Hist(hist.axis.Regular(60, 0, 12, name="TWC_TOA", label=r'Time Walk Corrected $\Delta$TOA [ns]'))
corrTOA_b3 = hist.Hist(hist.axis.Regular(60, 0, 12, name="TWC_TOA", label=r'Time Walk Corrected $\Delta$TOA [ns]'))

corrTOA_b0.fill(corr_toas[0])
corrTOA_b1.fill(corr_toas[1])
corrTOA_b3.fill(corr_toas[2])

fig, axes = plt.subplots(1, 3, figsize=(30, 10))
hep.cms.text(loc=0, ax=axes[0], text="Preliminary", fontsize=25)
axes[0].set_title(f'Board 0 Time Walk Correction', loc="right", size=25)
hep.histplot(corrTOA_b0, ax=axes[0], lw=2)
axes[0].set_xlabel('Time Walk Corrected TOA [ns]')
axes[0].set_ylabel('Entries')

hep.cms.text(loc=0, ax=axes[1], text="Preliminary", fontsize=25)
axes[1].set_title(f'Board 1 Time Walk Correction', loc="right", size=25)
hep.histplot(corrTOA_b1, ax=axes[1], lw=2)
axes[1].set_xlabel('Time Walk Corrected TOA [ns]')
axes[1].set_ylabel('Entries')

hep.cms.text(loc=0, ax=axes[2], text="Preliminary", fontsize=25)
axes[2].set_title(f'Board 3 Time Walk Correction', loc="right", size=25)
hep.histplot(corrTOA_b3, ax=axes[2], lw=2)
axes[2].set_xlabel('Time Walk Corrected TOA [ns]')
axes[2].set_ylabel('Entries')

plt.tight_layout()

In [ ]:
tmp_dict = {
    'evt': df_in_time['evt'].values,
    'corr_toa_b0': corr_toas[0],
    'corr_toa_b1': corr_toas[1],
    'corr_toa_b3': corr_toas[2],
}

df_in_time_corr = pd.DataFrame(tmp_dict)
del tmp_dict
df_in_time_corr

In [ ]:
diff_b01 = df_in_time_corr['corr_toa_b0'] - df_in_time_corr['corr_toa_b1']
diff_b03 = df_in_time_corr['corr_toa_b0'] - df_in_time_corr['corr_toa_b3']
diff_b13 = df_in_time_corr['corr_toa_b1'] - df_in_time_corr['corr_toa_b3']

dTOA_b01 = hist.Hist(hist.axis.Regular(80, diff_b01.mean().round(2)-0.8, diff_b01.mean().round(2)+0.8, name="TWC_TOA", label=r'Time Walk Corrected $\Delta$TOA [ns]'))
dTOA_b03 = hist.Hist(hist.axis.Regular(80, diff_b03.mean().round(2)-0.8, diff_b03.mean().round(2)+0.8, name="TWC_TOA", label=r'Time Walk Corrected $\Delta$TOA [ns]'))
dTOA_b13 = hist.Hist(hist.axis.Regular(80, diff_b13.mean().round(2)-0.8, diff_b13.mean().round(2)+0.8, name="TWC_TOA", label=r'Time Walk Corrected $\Delta$TOA [ns]'))

dTOA_b01.fill(diff_b01)
dTOA_b03.fill(diff_b03)
dTOA_b13.fill(diff_b13)

### Fit using lmfit ExponentialGaussianModel

In [ ]:
fit_params_lmfit = []
params = helper.lmfit_gaussfit_with_pulls(diff_b01, dTOA_b01, std_range_cut=0.4, width_factor=1, fig_title='Board 0 - Board 1', use_pred_uncert=True)
fit_params_lmfit.append(params)
params = helper.lmfit_gaussfit_with_pulls(diff_b03, dTOA_b03, std_range_cut=0.4, width_factor=1, fig_title='Board 0 - Board 3', use_pred_uncert=True)
fit_params_lmfit.append(params)
params = helper.lmfit_gaussfit_with_pulls(diff_b13, dTOA_b13, std_range_cut=0.4, width_factor=1, fig_title='Board 1 - Board 3', use_pred_uncert=True)
fit_params_lmfit.append(params)

In [ ]:
res_b0,err_b0 = helper.return_resolution_ps(fit_params_lmfit[0][0], fit_params_lmfit[0][1],
                                            fit_params_lmfit[1][0], fit_params_lmfit[1][1],
                                            fit_params_lmfit[2][0], fit_params_lmfit[2][1])
res_b1,err_b1 = helper.return_resolution_ps(fit_params_lmfit[0][0], fit_params_lmfit[0][1],
                                            fit_params_lmfit[2][0], fit_params_lmfit[2][1],
                                            fit_params_lmfit[1][0], fit_params_lmfit[1][1])
res_b3,err_b3 = helper.return_resolution_ps(fit_params_lmfit[1][0], fit_params_lmfit[1][1],
                                            fit_params_lmfit[2][0], fit_params_lmfit[2][1],
                                            fit_params_lmfit[0][0], fit_params_lmfit[0][1])

print(f'Board 0: {res_b0:.2f} ps, error: {err_b0:.2f} ps')
print(f'Board 1: {res_b1:.2f} ps, error: {err_b1:.2f} ps')
print(f'Board 3: {res_b3:.2f} ps, error: {err_b3:.2f} ps')

## Optimized Time Walk Correction -  Pairwise Neural Network

In [ ]:
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import initializers
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.layers import Dense, Input

In [ ]:
def return_dense_model(numpars=2, print_summary=False):
    input  = Input(shape=(numpars,), name='input')
    dense1 = Dense(8, activation='relu', name='dense1',kernel_initializer=initializers.RandomNormal(),bias_initializer=initializers.Zeros())(input)
    # dense2 = Dense(4, activation='relu', name='dense2',kernel_initializer=initializers.RandomNormal(),bias_initializer=initializers.Zeros())(dense1)
    output = Dense(1, activation='linear', name='output',kernel_initializer=initializers.RandomNormal(),bias_initializer=initializers.Zeros())(dense1)
    model  = Model(inputs=[input], outputs=output, name="simple_dense_NN")
    model.compile(loss='mse', optimizer='adam')
    if(print_summary): print(model.summary())
    return model

model = return_dense_model(print_summary=True)

del model

In [ ]:
ensemble_count = 10
data_tag = 'tb_sep28_offset24_triggeroffset6'
extra_tag = "CALNarrow_TOA275to350forWB_NoTOTcut"

In [ ]:
def etroc_regression_using_NN(
        input_df: pd.DataFrame,
        variables: list,
        data_tag: str,
        extra_tag: str,
        board_tag: str,
        ensemble_count: int,
        figure_title: str,
        do_plotting: bool,
    ):
    filename = f'{data_tag}_weights_{extra_tag}_{board_tag}'

    for en_idx in range(ensemble_count):
        model = return_dense_model(numpars=2)
        checkpointer = ModelCheckpoint(f'models/NNRun{en_idx}_{filename}.hdf5', verbose=0, save_best_only=True, monitor="val_loss")
        term = tf.keras.callbacks.TerminateOnNaN()
        escb = tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=15, verbose=0)

        shuffled_df = input_df.sample(frac=1)
        
        history = model.fit(
            shuffled_df[[variables[0], variables[1]]].values, 
            (shuffled_df[variables[2]]-shuffled_df[variables[3]]).values, 
            validation_split=0.3, 
            epochs=150,
            callbacks=[checkpointer,term,escb],
            verbose=0)
        
        del model

        if (do_plotting):
            #plot the loss and validation loss of the dataset
            fig, ax = plt.subplots(figsize=(15, 10), dpi=50)
            hep.cms.text(loc=0, ax=ax, text="Preliminary", fontsize=25)
            ax.set_title(f'Model {en_idx}, {figure_title}', loc="right", size=25)
            plt.plot(history.history['loss'], label='Train data')
            plt.plot(history.history['val_loss'], label='Validation data')
            plt.ylabel('Loss (MSE)')
            plt.yscale("log")
            plt.legend()
            plt.savefig(f"models/NNRun{en_idx}_{filename}.png")
            plt.show()

### Run NN Training (skip if you don't want to do training)

In [ ]:
vars = ['tot_b0', 'tot_b1', 'toa_b0', 'toa_b1']
etroc_regression_using_NN(df_in_time, vars, data_tag, extra_tag, "b01", ensemble_count, figure_title='Board 0 - Board 1', do_plotting=True)

vars = ['tot_b0', 'tot_b3', 'toa_b0', 'toa_b3']
etroc_regression_using_NN(df_in_time, vars, data_tag, extra_tag, "b03", ensemble_count, figure_title='Board 0 - Board 3', do_plotting=True)

vars = ['tot_b1', 'tot_b3', 'toa_b1', 'toa_b3']
etroc_regression_using_NN(df_in_time, vars, data_tag, extra_tag, "b13", ensemble_count, figure_title='Board 1 - Board 3', do_plotting=True)

### Call trained models

In [ ]:
model_b01 = return_dense_model(numpars=2)
filename = f'{data_tag}_weights_{extra_tag}_b01'
for en_idx in range(ensemble_count):
    model_b01.load_weights(f'models/NNRun{en_idx}_{filename}.hdf5')
    if(en_idx==0): Y_pred = model_b01.predict(df_in_time[['tot_b0', 'tot_b1']].values, verbose=0).flatten()
    else: Y_pred += model_b01.predict(df_in_time[['tot_b0', 'tot_b1']].values, verbose=0).flatten()
del model_b01
Y_pred_b01 = Y_pred/ensemble_count

model_b03 = return_dense_model(numpars=2)
filename = f'{data_tag}_weights_{extra_tag}_b03'
for en_idx in range(ensemble_count):
    model_b03.load_weights(f'models/NNRun{en_idx}_{filename}.hdf5')
    if(en_idx==0): Y_pred = model_b03.predict(df_in_time[['tot_b0', 'tot_b3']].values, verbose=0).flatten()
    else: Y_pred += model_b03.predict(df_in_time[['tot_b0', 'tot_b3']].values, verbose=0).flatten()
del model_b03
Y_pred_b03 = Y_pred/ensemble_count

model_b13 = return_dense_model(numpars=2)
filename = f'{data_tag}_weights_{extra_tag}_b13'
for en_idx in range(ensemble_count):
    model_b13.load_weights(f'models/NNRun{en_idx}_{filename}.hdf5')
    if(en_idx==0): Y_pred = model_b13.predict(df_in_time[['tot_b1', 'tot_b3']].values, verbose=0).flatten()
    else: Y_pred += model_b13.predict(df_in_time[['tot_b1',  'tot_b3']].values, verbose=0).flatten()
del model_b13
Y_pred_b13 = Y_pred/ensemble_count

In [ ]:
data_b01 = (df_in_time['toa_b0']-df_in_time['toa_b1']).values-Y_pred_b01
data_b03 = (df_in_time['toa_b0']-df_in_time['toa_b3']).values-Y_pred_b03
data_b13 = (df_in_time['toa_b1']-df_in_time['toa_b3']).values-Y_pred_b13

dTOA_NN_b01 = hist.Hist(hist.axis.Regular(50, data_b01.mean().round(2)-0.5, data_b01.mean().round(2)+0.5, name="TWC_TOA", label=r'Time Walk Corrected $\Delta$TOA [ns]'))
dTOA_NN_b03 = hist.Hist(hist.axis.Regular(50, data_b03.mean().round(2)-0.5, data_b03.mean().round(2)+0.5, name="TWC_TOA", label=r'Time Walk Corrected $\Delta$TOA [ns]'))
dTOA_NN_b13 = hist.Hist(hist.axis.Regular(50, data_b13.mean().round(2)-0.5, data_b13.mean().round(2)+0.5, name="TWC_TOA", label=r'Time Walk Corrected $\Delta$TOA [ns]'))

dTOA_NN_b01.fill(data_b01)
dTOA_NN_b03.fill(data_b03)
dTOA_NN_b13.fill(data_b13)

### Fit using lmfit ExponentialGaussianModel with NN outputs

In [ ]:
fit_params_lmfit_NN = []
params = helper.lmfit_gaussfit_with_pulls(data_b01, dTOA_NN_b01, std_range_cut=0.4, width_factor=1, fig_title='Board 0 - Board 1', use_pred_uncert=True)
fit_params_lmfit_NN.append(params)
params = helper.lmfit_gaussfit_with_pulls(data_b03, dTOA_NN_b03, std_range_cut=0.4, width_factor=1, fig_title='Board 0 - Board 3', use_pred_uncert=True)
fit_params_lmfit_NN.append(params)
params = helper.lmfit_gaussfit_with_pulls(data_b13, dTOA_NN_b13, std_range_cut=0.4, width_factor=1, fig_title='Board 1 - Board 3', use_pred_uncert=True)
fit_params_lmfit_NN.append(params)

In [ ]:
res_b0_NN,err_b0_NN = helper.return_resolution_ps(fit_params_lmfit_NN[0][0], fit_params_lmfit_NN[0][1],
                                            fit_params_lmfit_NN[1][0], fit_params_lmfit_NN[1][1],
                                            fit_params_lmfit_NN[2][0], fit_params_lmfit_NN[2][1])
res_b1_NN,err_b1_NN = helper.return_resolution_ps(fit_params_lmfit_NN[0][0], fit_params_lmfit_NN[0][1],
                                            fit_params_lmfit_NN[2][0], fit_params_lmfit_NN[2][1],
                                            fit_params_lmfit_NN[1][0], fit_params_lmfit_NN[1][1])
res_b3_NN,err_b3_NN = helper.return_resolution_ps(fit_params_lmfit_NN[1][0], fit_params_lmfit_NN[1][1],
                                            fit_params_lmfit_NN[2][0], fit_params_lmfit_NN[2][1],
                                            fit_params_lmfit_NN[0][0], fit_params_lmfit_NN[0][1])

print(f'Board 0: {res_b0_NN:.2f} ps, error: {err_b0_NN:.2f} ps')
print(f'Board 1: {res_b1_NN:.2f} ps, error: {err_b1_NN:.2f} ps')
print(f'Board 3: {res_b3_NN:.2f} ps, error: {err_b3_NN:.2f} ps')

## Make final result plot

In [ ]:
names = ['Top', 'Middle', 'Bottom']

board_resols = {
    'Single board TWC': ((res_b1, res_b0, res_b3), (err_b1, err_b0, err_b3)),
    'Pairwise TWC': ((res_b1_NN, res_b0_NN, res_b3_NN), (err_b1_NN, err_b0_NN, err_b3_NN))
}

x = np.arange(len(names))  # the label locations
width = 0.1  # the width of the bars
multiplier = 0

fig, ax = plt.subplots(figsize=(13, 6), layout='constrained')

for attribute, measurement in board_resols.items():
    offset = width * multiplier
    rects = ax.barh(x + offset, measurement[0], width, edgecolor='white', xerr=measurement[1], label=attribute)
    multiplier += 1

rectangle = mpatch.Rectangle((55, 0), 40, 0.7, edgecolor='black', facecolor="none", linewidth=1.5)
ax.add_patch(rectangle)
rx, ry = rectangle.get_xy()
cx = rx + rectangle.get_width()/2.0
cy = ry + rectangle.get_height()/2.
ax.annotate("Top: FFF corner, Wire bonded 2x2, W36-IP7-13 (HPK, split3) \n Middle: FFF corner, Bump bonded 16x16, W13 4-5 (FBK) \n Bottom: FFF corner, Bump bonded 16x16, W16 P6 (HPK)", (cx, cy), color='black', weight='bold', fontsize=13, ha='center', va='center')


# Add some text for labels, title and custom x-axis tick labels, etc.
hep.cms.text(loc=0, ax=ax, text="Preliminary", fontsize=25)
ax.set_title(rf'TOA$_{{{names[0]}}} \in \left[275, 350\right]$', loc="right", size=20)
ax.set_xlabel('Time Resolution [ps]', fontsize=20)  
ax.set_yticks(x + width, names)
ax.legend(loc='lower right', fontsize=18)
ax.set_xlim(30, 100)
ax.invert_yaxis()